In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [9]:
import pickle
import pandas as pd
import numpy

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [4]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [5]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')

In [6]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [10]:
df['predicted_duration'] = y_pred

In [12]:
df.describe()

,VendorID,passenger_count,trip_distance,RatecodeID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration,predicted_duration
count,2.918187e+06,2.822437e+06,2.918187e+06,2.822437e+06,2.918187e+06,2.918187e+06,2.918187e+06,2.918187e+06,2.918187e+06,2.918187e+06,2.918187e+06,2.918187e+06,2.822437e+06,2.822437e+06,2.918187e+06,2.918187e+06
mean,1.702265e+00,1.391660e+00,5.747751e+00,1.350555e+00,1.168508e+00,1.268422e+01,1.021348e+00,4.942594e-01,2.463432e+00,3.529895e-01,2.975203e-01,1.900824e+01,2.325968e+00,6.890632e-02,1.379372e+01,1.251342e+01
std,4.693961e-01,9.841901e-01,6.432019e+02,5.562606e+00,4.829941e-01,1.064428e+01,1.224293e+00,6.986270e-02,2.596137e+00,1.592588e+00,3.836547e-02,1.346242e+01,6.728310e-01,2.865852e-01,9.643098e+00,5.281404e+00
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,-6.000000e+02,-4.500000e+00,-5.000000e-01,-1.880000e+02,-2.300000e+01,-3.000000e-01,-6.003000e+02,-2.500000e+00,-1.250000e+00,1.000000e+00,-7.485578e+00
25%,1.000000e+00,1.000000e+00,1.100000e+00,1.000000e+00,1.000000e+00,6.500000e+00,0.000000e+00,5.000000e-01,1.000000e+00,0.000000e+00,3.000000e-01,1.180000e+01,2.500000e+00,0.000000e+00,7.033333e+00,9.945231e+00
50%,2.000000e+00,1.000000e+00,1.800000e+00,1.000000e+00,1.000000e+00,9.500000e+00,5.000000e-01,5.000000e-01,2.060000e+00,0.000000e+00,3.000000e-01,1.518000e+01,2.500000e+00,0.000000e+00,1.126667e+01,1.066465e+01
75%,2.000000e+00,1.000000e+00,3.150000e+00,1.000000e+00,1.000000e+00,1.450000e+01,2.500000e+00,5.000000e-01,3.090000e+00,0.000000e+00,3.000000e-01,2.075000e+01,2.500000e+00,0.000000e+00,1.766667e+01,1.213280e+01
max,6.000000e+00,9.000000e+00,3.487985e+05,9.900000e+01,5.000000e+00,6.500000e+02,1.030000e+01,3.300000e+00,3.808000e+02,9.500000e+01,3.000000e-01,6.503000e+02,2.750000e+00,1.250000e+00,6.000000e+01,8.013667e+01


In [13]:
month = 2
year = 2022
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [15]:
result_df = df[['ride_id', 'predicted_duration']]

In [17]:
output_file = f"results_{year:04d}-{month:02d}"
result_df.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [18]:
!ls -l --block-size=M

total 58M
-rw-rw-r-- 1 inris inris  1M Jun 21 20:37 Dockerfile
-rw-rw-r-- 1 inris inris  1M Jun 27 13:07 Pipfile
-rw-r--r-- 1 inris inris  1M Jun 27 13:05 Pipfile.lock
-rw-rw-r-- 1 inris inris  1M Jun 21 20:37 model.bin
-rw-rw-r-- 1 inris inris 58M Jun 27 13:27 results_2022-02
-rw-rw-r-- 1 inris inris  1M Jun 27 13:28 starter.ipynb


In [20]:
!jupyter nbconvert --to script starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to script
[NbConvertApp] Writing 1459 bytes to starter.py
